In [110]:
# Imports
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [111]:
main_df = pd.read_csv('../data/processed/PROCESSED_DF.csv')
main_df.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,55601,6787,7637,6878,7089,7582,7738,5826,4050,2014,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,218022,24757,26913,23579,25213,27338,29986,29932,20936,9368,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,24881,2732,2960,3268,3201,3074,3278,3076,2244,1048,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,22400,2456,2596,3029,3113,3038,3115,2545,1723,785,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,57840,7095,7570,6742,6884,7474,7844,6965,4931,2335,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [112]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(main_df)

In [113]:
target = ['anycondition_prevalence', 'Obesity_prevalence', 'Heart disease_prevalence', 'COPD_prevalence', 'diabetes_prevalence', 'CKD_prevalence']
X = main_df.drop(columns=target)
features = X.columns.tolist()

X = main_df[features]
y = main_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size = 0.80)

In [114]:
lasso_model = Lasso(alpha=0.1, max_iter=300)
lasso_model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.600e+02, tolerance: 1.022e+01
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.552e+02, tolerance: 5.108e+00
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2

Lasso(alpha=0.1, max_iter=300)

In [115]:
y_pred = lasso_model.predict(X_test)
y_pred

array([[46.62627485, 37.0173956 ,  9.16206878,  9.57864058, 14.66258126,
         3.75075766],
       [39.90984191, 29.73691385,  6.39135763,  5.88680103, 10.17406607,
         2.85387546],
       [64.06825386, 43.43346603, 12.77890486, 16.55907908, 19.06582769,
         4.48031235],
       ...,
       [47.73406096, 37.01874769,  8.54491601,  9.37985423, 12.29873103,
         3.45592702],
       [52.65460982, 39.04828913,  8.72428928,  9.51799224, 15.04684237,
         3.65904483],
       [44.11138713, 34.03974621,  7.05257464,  6.41541368, 11.15229023,
         2.7636982 ]])

In [116]:
print(f"Error cuadrático medio: {mean_squared_error(y_test, y_pred)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.015782668676916495
Coeficiente de determinación: 0.9821477729037825


In [117]:
y_pred_train = lasso_model.predict(X_train)
y_pred_train

array([[49.59953938, 38.57211177,  9.95134548, 11.25085382, 13.76293853,
         3.59382814],
       [46.57597728, 33.204151  ,  9.14224754,  9.9450454 , 13.05671703,
         3.56022071],
       [47.01643467, 33.76950082,  7.45841351,  9.02640933, 12.14388087,
         3.12322598],
       ...,
       [52.9912351 , 39.00867775, 10.05343272, 12.21775148, 15.95149365,
         3.99855959],
       [52.29883899, 36.20696038, 12.76872491, 13.0255905 , 16.10800125,
         4.17349603],
       [46.87677911, 36.49263717,  8.03467666,  7.09758125, 10.73001213,
         3.14184005]])

In [118]:
print(f"Error cuadrático medio: {mean_squared_error(y_train, y_pred_train)}")
print(f"Coeficiente de determinación: {r2_score(y_train, y_pred_train)}")

Error cuadrático medio: 0.013639317627817648
Coeficiente de determinación: 0.9831333547449921


In [119]:
scores = cross_val_score(lasso_model, X, y, cv=5, scoring='r2')
print("R2 Promedio:", scores.mean())

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.598e+02, tolerance: 9.218e+00
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.546e+02, tolerance: 4.280e+00
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2

R2 Promedio: 0.9786729169273428


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.302e+01, tolerance: 8.391e-02
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.600e+02, tolerance: 1.000e+01
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2

In [120]:
ridge_model = Ridge(alpha=0.1, max_iter=300)
ridge_model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.1897e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(alpha=0.1, max_iter=300)

In [121]:
y_pred = ridge_model.predict(X_test)
y_pred

array([[46.54277637, 37.01391674,  9.32891029,  9.76441718, 14.79956174,
         3.93246997],
       [39.934338  , 29.71527634,  6.46505502,  5.95578552, 10.19404282,
         2.80300505],
       [64.10972352, 43.47559185, 13.02386126, 16.92027776, 19.10920625,
         4.34094211],
       ...,
       [47.80173972, 37.06259663,  8.4796725 ,  9.31572066, 12.20857029,
         3.34291517],
       [52.64458524, 39.05906802,  8.67375838,  9.51706597, 15.05158082,
         3.495181  ],
       [44.13887678, 34.05642441,  7.12381717,  6.30843293, 11.19095241,
         2.88609567]])

In [122]:
print(f"Error cuadrático medio: {mean_squared_error(y_test, y_pred)}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.0027060643967024398
Coeficiente de determinación: 0.9990296893355528


In [123]:
y_pred_train = ridge_model.predict(X_train)
y_pred_train

array([[49.66860044, 38.62605662, 10.07705782, 11.25093454, 13.76290897,
         3.78241326],
       [46.66657995, 33.22170035,  9.11875258,  9.8572334 , 13.0603308 ,
         3.49477403],
       [47.03498761, 33.7749778 ,  7.32336399,  9.1009797 , 12.14342356,
         2.89203781],
       ...,
       [52.94861864, 39.02622565, 10.04170897, 12.36736276, 15.95842773,
         4.03271926],
       [52.44703049, 36.22684214, 13.05486703, 13.03929021, 16.16427011,
         4.57702528],
       [46.89171511, 36.51848331,  7.8765203 ,  6.89490067, 10.60257931,
         3.12539591]])

In [124]:
print(f"Error cuadrático medio: {mean_squared_error(y_train, y_pred_train)}")
print(f"Coeficiente de determinación: {r2_score(y_train, y_pred_train)}")

Error cuadrático medio: 0.0024142415560476685
Coeficiente de determinación: 0.9991096460263115


In [125]:
scores = cross_val_score(ridge_model, X, y, cv=5, scoring='r2')
print("R2 Promedio:", scores.mean())

R2 Promedio: 0.9989433504191751


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.79324e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.34474e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.04087e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.31687e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.10/site-packages/skl